In [84]:
import matplotlib
matplotlib.use("GTKAgg")
import matplotlib.pyplot as plt
%matplotlib inline


import numpy as np
from sklearn import svm, datasets
from sklearn.metrics import accuracy_score

import os
import string

from PIL import Image

In [100]:
# CODE TO OBTAIN PATHS CORRESPONDING TO FEATURE ACTION UNIT CODES OF EACH SEQUENCE

lst = []
lst1=[]
#C:\\Users\\Harshit\\Downloads\\minor_project\\FACS
for i in os.walk("C:\\Rhea\\minor_project\\FACS"):
    #print i
    if i not in lst :
        if(len(i[2])>0):
            lst.append(str(i[0]) + "\\"+str(i[2][0]))
        else:
            #print i[0],i[1],i[2]
            lst1.append(str(i[0])+"\\"+str(i[1][0]))
            
print lst1, len(lst)

['C:\\Rhea\\minor_project\\FACS\\S005', 'C:\\Rhea\\minor_project\\FACS\\S005\\001', 'C:\\Rhea\\minor_project\\FACS\\S010\\001', 'C:\\Rhea\\minor_project\\FACS\\S011\\001', 'C:\\Rhea\\minor_project\\FACS\\S014\\001', 'C:\\Rhea\\minor_project\\FACS\\S022\\001', 'C:\\Rhea\\minor_project\\FACS\\S026\\001', 'C:\\Rhea\\minor_project\\FACS\\S028\\001', 'C:\\Rhea\\minor_project\\FACS\\S029\\001', 'C:\\Rhea\\minor_project\\FACS\\S032\\001', 'C:\\Rhea\\minor_project\\FACS\\S034\\001', 'C:\\Rhea\\minor_project\\FACS\\S035\\001', 'C:\\Rhea\\minor_project\\FACS\\S037\\001', 'C:\\Rhea\\minor_project\\FACS\\S042\\001', 'C:\\Rhea\\minor_project\\FACS\\S044\\001', 'C:\\Rhea\\minor_project\\FACS\\S045\\001', 'C:\\Rhea\\minor_project\\FACS\\S046\\001', 'C:\\Rhea\\minor_project\\FACS\\S050\\001', 'C:\\Rhea\\minor_project\\FACS\\S051\\001', 'C:\\Rhea\\minor_project\\FACS\\S052\\001', 'C:\\Rhea\\minor_project\\FACS\\S053\\001', 'C:\\Rhea\\minor_project\\FACS\\S054\\001', 'C:\\Rhea\\minor_project\\FACS\\S055

In [99]:
# CODE TO FIND OUT ALL THE ACTION-UNITS AVAILABLE IN DATASET

dic={}
for i in range(1,65):
    if i not in dic:
        dic[i]=0
for x in lst:
    with open(x) as f:
        lines = f.read().split()
        for i in range(0,len(lines)-1,2):
            dic[int(float(lines[i]))]+=1
au_nos = []
for a in dic.keys():
    if dic[a] > 0:
        au_nos.append(a)

print au_nos, len(au_nos)

[1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 38, 39, 43, 44, 45, 54, 61, 62, 63, 64] 39


In [123]:
# CODE TO ACCESS EMOTION LABELS OF ALL SEQUENCES: UNLABELED SEQUENCES ARE TREATED AS TEST CASES

# dic={1:0,2:0,4:0,5:0,6:0,7:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,20:0,21:0,23:0,24:0,25:0,26:0,27:0,28:0,
#      29:0,31:0,34:0,38:0,39:0,43:0,45:0}

ctr = 0
training = []
test = []
cnt=0
test_pics = []
for x in lst:
    dic={}
    for i in range(1,65):
        if i not in dic:
            dic[i]=0
    with open(x) as f:
        flag=0
        y=string.replace(x,"FACS","Emotion")
        y=string.replace(y,"facs","emotion")
        try:
            with open(y) as emo:
                e=emo.read()
                e=int(float(e))
                #print e
        except:
            flag=1
            test_pics.append(y)
            
        lines = f.read().split()
        for i in range(0,len(lines)-1,2):
            y = 1 if (int(float(lines[i+1]))==0) else int(float(lines[i+1]))
            dic[int(float(lines[i]))]+=y
    features = []
    #print dic.values()
    for a in au_nos:
        features.append(dic[a])
    #print features
    if(flag==0):
        training.append([features, e])
    else:
        test.append([features, 0])

print len(test_pics), len(training)
print test_pics[:3]


266 327
['C:\\Rhea\\minor_project\\Emotion\\S010\\001\\S010_001_00000014_emotion.txt', 'C:\\Rhea\\minor_project\\Emotion\\S010\\003\\S010_003_00000018_emotion.txt', 'C:\\Rhea\\minor_project\\Emotion\\S010\\005\\S010_005_00000016_emotion.txt']


In [97]:
# SAMPLES OF EACH EMOTION SEPARATED INTO 5 FOLDS EACH SUCH THAT EACH FOLD MAINTAINS THE SAME DISTRIBUTION AS THE ENIRE DATASET. 

all_folds = []
all_emos = []

#ANGER: {(57, 9), (57, 9), (56, 9), (56, 9), (56, 9)}
y = 0
pos = []
neg = []
an_test = []

for x in training:
    if x[-1]!=1:
        neg.append([x[0],0])
    else:
        pos.append([x[0],1])
        y += 1

an_test.append(np.array(neg[:57]+ pos[:9]))
an_test.append(np.array(neg[57:114]+ pos[9:18]))
an_test.append(np.array(neg[114:170]+ pos[18:27]))
an_test.append(np.array(neg[170:226]+ pos[27:36]))
an_test.append(np.array(neg[226:282]+ pos[36:45]))
an_test = np.array(an_test)
print  an_test[0].shape, an_test[1].shape, an_test[2].shape, an_test[3].shape, an_test[4].shape
print y
all_folds.append(an_test)
all_emos.append(pos+neg)

#CONTEMPT: {(62 ,3), (62 ,3), (62 ,4), (62 ,4), (61 ,4)}
y = 0
pos = []
neg = []
an_test = []

for x in training:
    if x[-1]!=2:
        neg.append([x[0],0])
    else:
        pos.append([x[0],1])
        y += 1

an_test.append(np.array(pos[:3]+ neg[:62]))
an_test.append(np.array(pos[3:6]+ neg[62:124]))
an_test.append(np.array(pos[6:10]+ neg[124:186]))
an_test.append(np.array(pos[10:14]+ neg[186:248]))
an_test.append(np.array(pos[14:18]+ neg[248:309]))
an_test = np.array(an_test)
print  an_test[0].shape, an_test[1].shape, an_test[2].shape, an_test[3].shape, an_test[4].shape
print y
all_folds.append(an_test)
all_emos.append(pos+neg)


#DISGUST: {(54 ,11), (54 ,12), (54 ,12), (53 ,12), (53 ,12)}
y = 0
pos = []
neg = []
an_test = []
for x in training:
    if x[-1]!=3:
        neg.append([x[0],0])
    else:
        pos.append([x[0],1])
        y += 1
an_test.append(np.array(pos[:11]+ neg[:54]))
an_test.append(np.array(pos[11:23]+ neg[54:108]))
an_test.append(np.array(pos[23:35]+ neg[108:162]))
an_test.append(np.array(pos[35:47]+ neg[162:215]))
an_test.append(np.array(pos[47:59]+ neg[215:268]))
an_test = np.array(an_test)
print  an_test[0].shape, an_test[1].shape, an_test[2].shape, an_test[3].shape, an_test[4].shape
print y
all_emos.append(pos+neg)
all_folds.append(an_test)


#FEAR: {(61 ,5), (61 ,5), (60 ,5), (60 ,5), (60 ,5)}
y = 0
pos = []
neg = []
an_test = []
for x in training:
    if x[-1]!=4:
        neg.append([x[0],0])
    else:
        pos.append([x[0],1])
        y += 1

an_test.append(np.array(pos[:5]+ neg[:61]))
an_test.append(np.array(pos[5:10]+ neg[61:122]))
an_test.append(np.array(pos[10:15]+ neg[122:182]))
an_test.append(np.array(pos[15:20]+ neg[182:242]))
an_test.append(np.array(pos[20:25]+ neg[242:302]))
an_test = np.array(an_test)
print  an_test[0].shape, an_test[1].shape, an_test[2].shape, an_test[3].shape, an_test[4].shape
print y
all_emos.append(pos+neg)
all_folds.append(an_test)
print len(all_emos)


(66L, 2L) (66L, 2L) (65L, 2L) (65L, 2L) (65L, 2L)
45
(65L, 2L) (65L, 2L) (66L, 2L) (66L, 2L) (65L, 2L)
18
(65L, 2L) (66L, 2L) (66L, 2L) (65L, 2L) (65L, 2L)
59
(66L, 2L) (66L, 2L) (65L, 2L) (65L, 2L) (65L, 2L)
25
4


In [98]:
# SAMPLES OF EACH EMOTION SEPARATED INTO 5 FOLDS EACH SUCH THAT EACH FOLD MAINTAINS THE SAME DISTRIBUTION AS THE ENIRE DATASET. 

#HAPPY: {(51 ,14), (51 ,14), (52 ,14), (52 ,14), (52 ,13)}
y = 0
pos = []
neg = []
an_test = []
for x in training:
    if x[-1]!=5:
        neg.append([x[0],0])
    else:
        pos.append([x[0],1])
        y += 1
        
an_test.append(np.array(pos[:14]+ neg[:51]))
an_test.append(np.array(pos[14:28]+ neg[51:102]))
an_test.append(np.array(pos[28:42]+ neg[102:154]))
an_test.append(np.array(pos[42:56]+ neg[154:206]))
an_test.append(np.array(pos[56:69]+ neg[206:258]))
an_test = np.array(an_test)
print  an_test[0].shape, an_test[1].shape, an_test[2].shape, an_test[3].shape, an_test[4].shape
print y
all_emos.append(pos+neg)
all_folds.append(an_test)


#SADNESS: {(59 ,6), (60 ,6), (60 ,6), (60 ,5), (60 ,5)}
y = 0
pos = []
neg = []
an_test = []
for x in training:
    if x[-1]!=6:
        neg.append([x[0],0])
    else:
        pos.append([x[0],1])
        y += 1
an_test.append(np.array(pos[:6]+ neg[:59]))
an_test.append(np.array(pos[6:12]+ neg[59:119]))
an_test.append(np.array(pos[12:18]+ neg[119:179]))
an_test.append(np.array(pos[18:23]+ neg[179:239]))
an_test.append(np.array(pos[23:28]+ neg[239:299]))
an_test = np.array(an_test)
print  an_test[0].shape, an_test[1].shape, an_test[2].shape, an_test[3].shape, an_test[4].shape
print y
all_emos.append(pos+neg)
all_folds.append(an_test)

#SURPRISE: {(48 ,17), (49 ,17), (49 ,17), (49 ,16), (49 ,16)}
y = 0
pos = []
neg = []
an_test = []
for x in training:
    if x[-1]!=7:
        neg.append([x[0],0])
    else:
        pos.append([x[0],1])
        y += 1

an_test.append(np.array(pos[:17]+ neg[:48]))
an_test.append(np.array(pos[17:34]+ neg[48:97]))
an_test.append(np.array(pos[34:51]+ neg[97:146]))
an_test.append(np.array(pos[51:67]+ neg[146:195]))
an_test.append(np.array(pos[67:83]+ neg[195:244]))
an_test = np.array(an_test)
print  an_test[0].shape, an_test[1].shape, an_test[2].shape, an_test[3].shape, an_test[4].shape
print y
all_emos.append(pos+neg)
all_folds.append(an_test)

all_folds = np.array(all_folds)
all_emos = np.array(all_emos)

print all_emo.shape, all_folds.shape

(65L, 2L) (65L, 2L) (66L, 2L) (66L, 2L) (65L, 2L)
69
(65L, 2L) (66L, 2L) (66L, 2L) (65L, 2L) (65L, 2L)
28
(65L, 2L) (66L, 2L) (66L, 2L) (65L, 2L) (65L, 2L)
83
(7L, 327L, 2L) (7L, 5L)


In [41]:
# SEVEN DIFFERENT CLASSIFIERS FOR EACH EMOTION ARE MADE, IN KEEPING WITH THE PROPERTY OF ONE VS REST CLASSIFICATION.
# EACH CLASSIFIER IN TURN IS CROSS VALIDATED USING K (=5) DIFFERENT CLASSIFIERS.
# A POLYNOMIAL OF DEGREE 3 IS USED AS A KERNEL FUNCTION

accu = 0.0
for emo in all_folds:
    #emo = all_folds[0] 
    tot_accu = 0.0
    print '\n------------------------NEXT EMOTION-------------------------------\n'
    for i in range(0, 5):
        print '\nWorking on fold #'+str(i) + ' as testing data'
        
        test_feat = []
        test_targ = []

        for o in emo[i]:
            test_feat.append(o[0])
            test_targ.append(o[1])

        train_feat = []
        train_targ = []

        for j in range(0, 5):
            if j != i:
                for o in emo[j]:
                    train_feat.append(o[0])
                    train_targ.append(o[1])           

        print len(test_feat), len(train_feat)

        clf = svm.SVC(kernel='poly', degree=3, C=C).fit(np.array(train_feat), np.array(train_targ)) #osumz
        
        #Checking on training data, i.e the single folds of data
        errors1 = 0
        Z = clf.predict(train_feat)
        
        for i in range(0, len(Z)):
            if (Z[i] != train_targ[i]):
                errors1 += 1
                
        print "Errors on training data are: ", errors1
        
        #Checking on testing data, i.e k-1 folds of data
        errors2 = 0
        Z = clf.predict(test_feat)
        
        for i in range(0, len(Z)):
            if (Z[i] != test_targ[i]):
                errors2 += 1
        print "Errors on testing data are: ", errors2
        fold_accu = 100.00 - float(errors2*100.00)/len(test_targ)
        tot_accu += fold_accu
        print "Accuracy here = " + str(fold_accu)
        
    tot_accu /= 5
    print '\nFor this emotion, accuracy is ' + str(tot_accu)
    accu += tot_accu
        
accu /= 7
print '\n\nFor a 3 degree polynomial as kernel function, accuracy is: ' + str(accu)


------------------------NEXT EMOTION-------------------------------


Working on fold #0 as testing data
66 261
Errors on training data are:  18
Errors on testing data are:  4
Accuracy here = 93.9393939394

Working on fold #1 as testing data
66 261
Errors on training data are:  18
Errors on testing data are:  5
Accuracy here = 92.4242424242

Working on fold #2 as testing data
65 262
Errors on training data are:  13
Errors on testing data are:  7
Accuracy here = 89.2307692308

Working on fold #3 as testing data
65 262
Errors on training data are:  18
Errors on testing data are:  4
Accuracy here = 93.8461538462

Working on fold #4 as testing data
65 262
Errors on training data are:  17
Errors on testing data are:  5
Accuracy here = 92.3076923077

For this emotion, accuracy is 92.3496503497

------------------------NEXT EMOTION-------------------------------


Working on fold #0 as testing data
65 262
Errors on training data are:  15
Errors on testing data are:  3
Accuracy here = 95.3846

In [105]:
# SEVEN DIFFERENT CLASSIFIERS FOR EACH EMOTION ARE MADE, IN KEEPING WITH THE PROPERTY OF ONE VS REST CLASSIFICATION.
# EACH CLASSIFIER IN TURN IS CROSS VALIDATED USING K (=5) DIFFERENT CLASSIFIERS.
# A POLYNOMIAL OF DEGREE 3 IS USED AS A KERNEL FUNCTION

accu = 0.0
for emo in all_folds:
    #emo = all_folds[0] 
    tot_accu = 0.0
    print '\n------------------------NEXT EMOTION-------------------------------\n'
    for i in range(0, 5):
        print '\nWorking on fold #'+str(i) + ' as testing data'
        
        test_feat = []
        test_targ = []

        for o in emo[i]:
            test_feat.append(o[0])
            test_targ.append(o[1])

        train_feat = []
        train_targ = []

        for j in range(0, 5):
            if j != i:
                for o in emo[j]:
                    train_feat.append(o[0])
                    train_targ.append(o[1])           

        print len(test_feat), len(train_feat)

        clf = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(np.array(train_feat), np.array(train_targ)) #osumz
        
        #Checking on training data, i.e the single folds of data
        errors1 = 0
        Z = clf.predict(train_feat)
        
        for i in range(0, len(Z)):
            if (Z[i] != train_targ[i]):
                errors1 += 1
                
        print "Errors on training data are: ", errors1
        
        #Checking on testing data, i.e k-1 folds of data
        errors2 = 0
        Z = clf.predict(test_feat)
        
        for i in range(0, len(Z)):
            if (Z[i] != test_targ[i]):
                errors2 += 1
        print "Errors on testing data are: ", errors2
        fold_accu = 100.00 - float(errors2*100.00)/len(test_targ)
        tot_accu += fold_accu
        print "Accuracy here = " + str(fold_accu)
        
    tot_accu /= 5
    print '\nFor this emotion, accuracy is ' + str(tot_accu)
    accu += tot_accu
        
accu /= 7
print '\n\nFor a 3 degree polynomial as kernel function, accuracy is: ' + str(accu)


------------------------NEXT EMOTION-------------------------------


Working on fold #0 as testing data
66 261
Errors on training data are:  0
Errors on testing data are:  8
Accuracy here = 87.8787878788

Working on fold #1 as testing data
66 261
Errors on training data are:  0
Errors on testing data are:  6
Accuracy here = 90.9090909091

Working on fold #2 as testing data
65 262
Errors on training data are:  0
Errors on testing data are:  7
Accuracy here = 89.2307692308

Working on fold #3 as testing data
65 262
Errors on training data are:  0
Errors on testing data are:  9
Accuracy here = 86.1538461538

Working on fold #4 as testing data
65 262
Errors on training data are:  0
Errors on testing data are:  6
Accuracy here = 90.7692307692

For this emotion, accuracy is 88.9883449883

------------------------NEXT EMOTION-------------------------------


Working on fold #0 as testing data
65 262
Errors on training data are:  1
Errors on testing data are:  3
Accuracy here = 95.3846153846

In [121]:
# CHECKING FOR ONE VS ONE PERFORMANCE OF SVM ON TRAINING DATA USING A 3:1 DIVISION

emoss = ["anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]

train_data = []
train_labels = []

x = int(3.0*327.0/4.0)

for t in training[:x]:
    train_data.append(t[0])
    train_labels.append(t[1])

clf = svm.SVC(kernel='poly', degree=3,  C=C).fit(np.array(train_data), np.array(train_labels))
err = 0.0

for t in training[x:]:
    
    testt = np.array(t[0]).reshape(1, -1)
    res = clf.predict(testt)
    
    if res[0] != t[1]:
        err += 1
        print 'Expected was: '+str(t[1]) + '  Result is: ' + str(res[0])

accuracy = 100.00 - float(err/3.27)
print accuracy

Expected was: 5  Result is: 7
Expected was: 3  Result is: 7
Expected was: 1  Result is: 7
Expected was: 5  Result is: 7
Expected was: 3  Result is: 7
Expected was: 5  Result is: 7
Expected was: 6  Result is: 7
Expected was: 3  Result is: 7
Expected was: 5  Result is: 7
Expected was: 6  Result is: 7
Expected was: 5  Result is: 7
Expected was: 6  Result is: 7
Expected was: 4  Result is: 7
Expected was: 3  Result is: 7
Expected was: 5  Result is: 7
Expected was: 1  Result is: 7
Expected was: 1  Result is: 7
Expected was: 5  Result is: 7
Expected was: 3  Result is: 7
Expected was: 5  Result is: 7
Expected was: 6  Result is: 7
Expected was: 1  Result is: 7
Expected was: 5  Result is: 7
Expected was: 6  Result is: 7
Expected was: 5  Result is: 7
Expected was: 4  Result is: 7
Expected was: 5  Result is: 7
Expected was: 6  Result is: 7
Expected was: 2  Result is: 7
Expected was: 2  Result is: 7
Expected was: 2  Result is: 7
Expected was: 2  Result is: 7
Expected was: 2  Result is: 7
Expected w

In [122]:
# CODE TO CHECK FOR THE PREDICTIONS MADE BY THE CLASSIFIERS FOR TEST DATA SET USING ONE VS REST APPROACH.
# IF LABEL IS NOT FOUND, SAMPLE IS LABELED NEUTRAL.

emoss = ["anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"]

def classifyy(feat_set):
    '''
    Uses ORing of results of k (=5) different cross validated classifiers. 
    Only a value of 1 is treated as a positive result for that particular emotion.
    '''
    emos_predicted = []
    for no,emo in enumerate(all_folds):
        val = False
        
        for i in range(0, 5):
            
            train_feat = []
            train_targ = []

            for j in range(0, 5):
                if j != i:
                    for o in emo[j]:
                        train_feat.append(o[0])
                        train_targ.append(o[1])           

            clf = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(np.array(train_feat), np.array(train_targ))  #osumz

            z = clf.predict(feat_set)
            val = val + z[0]
        
        if val:
            emos_predicted.append(emoss[no])

    return emos_predicted


n = 0
for k in range(0, len(test)):

    t = test[k]
    to_classify = np.array(t[0]).reshape(1, -1)
    result = classifyy(to_classify)
#     if (len(result) > 0):
#         #print test_pics[k][:38], result
#     else:
        #print test_pics[k][:38], 'neutral'
    
    if (len(result) > 1):
        n += 1
        
print n
    

1


In [77]:
print(len('C:\\Rhea\\minor_project\\Emotion\\S010\\001'))

38
